# Text Mining 예시

In [2]:
import pandas as pd
import os
import django
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from board.models import Board

all_posts = Board.objects.all()
all_posts_content = [post.content for post in all_posts]

# 모든 게시물의 단어 빈도 계산하는 로직
all_words = [word for content in all_posts_content for word in content.split()]
word_freq = Counter(all_words)

# 전체 게시물 수
total_documents = len(all_posts_content)
all_posts_content = pd.DataFrame([post.content for post in all_posts], index = [post.id for post in all_posts], columns= ["Content"])

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_df = 0.6) ## 60퍼센트 이상 나온 단어는 무시해
matrix = vectorizer.fit_transform(all_posts_content["Content"]) ## fit은 데이터를 학습시키는 메서드 transform은 학습시킨 것을 적용하는 메서드

# 단어와 인덱스를 매핑
feature_names = vectorizer.get_feature_names_out(matrix)
word_index_mapping = {word: index for index, word in enumerate(feature_names)}

# 40퍼 이하 빈도 단어 찾기
low_freq_words = [word for word, freq in word_freq.items() if (freq / total_documents) <= 0.4]

# 필터링된 단어로만 이루어진 문서 만들기
relevant_word_indices = [word_index_mapping[word] for word in low_freq_words if word in word_index_mapping]

# 게시물 간의 유사도 계산
cosine_similarities = cosine_similarity(matrix, matrix)

# 주어진 포스트와 다른 포스트의 연관성 평가
related_posts_index = cosine_similarities[31].argsort()[::-1][1:6]

related_posts = Board.objects.filter(id__in = related_posts_index)

for post in related_posts:
    print(f" - {post.title}: {post.content}")

# 각 문서의 ID, 단어들, 유사도 출력
related_posts = []
for i, post in enumerate(all_posts):
    words = [word for word in post.content.split()]
    similarity = cosine_similarities[1][i]
    if similarity > 0:
        related_posts.append(post.id)
        print(f"Post ID: {post.id}")
        print(f"Document {i} Words: {words}")
        print(f"Similarity with Query Document: {similarity}")
related_posts

def find_related_posts(post_id):
    posts = Board.objects.all()
    all_posts_content = [post.content for post in all_posts]

    related_posts = []
    return related_posts

find_related_posts(1)

ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.